# kNN+gzip is all you need?! 

A week ago or so I heard about this “Low-Resource” Text Classification: A Parameter-Free Classification
Method with Compressors paper (https://aclanthology.org/2023.findings-acl.426.pdf), where they claimed to beat BeRT on sentiment analysis classification. 

Once in a while, I see something and I am just compelled to try for myself, and this was one of those times. 

It has everything, it’s simple, it’s fast, it’s easy to implement, and it's my favorite machine learning algorithm, k Nearest Neighbors. I sometimes do contracting and consulting for clients, and the number of times that I've told someone that their problem doesn't need deep learning and can instead be done easier, cheaper and just as effectively, with a kNN approach, is too many to count. So, when I see something super simple like this, doing text classification for sentiment analysis with kNN for the classification and gzip with "compression distances" being used as the feature vectors, I just have to try it for myself.

The research paper came with code on github: https://github.com/bazingagin/npc_gzip, but it comes with a bunch of extra tweaks and techniques for getting the most performance, but I want something super simple and clear. I just wanted to confirm that I had a grasp on how this actually works, because this idea of compressed distances sounds simple but also a bit strange, so if you're a little fuzzy on what the heck this is, no worries because we're going to be "from scratch-ing" it here, though you should already know what k Nearest Neighbors is. I have an older tutorial on it where I built everything from scratch, if you do not know what kNN is, you can check that out here: https://pythonprogramming.net/k-nearest-neighbors-application-machine-learning-tutorial/

# Gzip method

In [40]:
%%time
import gzip
from sklearn.neighbors import KNeighborsClassifier
import pickle


N_SAMPLES = 500 

with open(f"sentiment-dataset-{N_SAMPLES}.pickle", "rb") as f:
    dataset = pickle.load(f)

train_x, train_y, test_x, test_y = dataset # samples are text strings (x) and sentiment -1 or 1 (y)

def ncd(x, x2): # NCD with compressed lengths
    x_compressed = len(gzip.compress(x.encode()))
    x2_compressed = len(gzip.compress(x2.encode()))  
    xx2 = len(gzip.compress((" ".join([x,x2])).encode()))
    return (xx2 - min(x_compressed, x2_compressed)) / max(x_compressed, x2_compressed)

train_ncd = [[ncd(train_x[i], train_x[j]) for j in range(len(train_x))] for i in range(len(train_x))]
test_ncd = [[ncd(test_x[i], train_x[j]) for j in range(len(train_x))] for i in range(len(test_x))]

# KNN classification
neigh = KNeighborsClassifier(n_neighbors=7) 
neigh.fit(train_ncd, train_y)
print("Accuracy:", neigh.score(test_ncd, test_y))

Accuracy: 0.7029702970297029
CPU times: user 36.8 s, sys: 14.4 ms, total: 36.8 s
Wall time: 36.9 s


# Traditional TF-IDF + Logistic regression
from https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html

In [42]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Extracting features from the training data using a sparse vectorizer
vectorizer = TfidfVectorizer(
    sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
)
X_train = vectorizer.fit_transform(train_x)
X_test = vectorizer.transform(test_x)
feature_names = vectorizer.get_feature_names_out()

clf = LogisticRegression(C=5, max_iter=1000)
clf.fit(X_train, train_y)
# pred = clf.predict(X_test)

# score = metrics.accuracy_score(test_y, pred)
print("Accuracy:", clf.score(X_test, test_y))

Accuracy: 0.8613861386138614
CPU times: user 108 ms, sys: 2.87 ms, total: 111 ms
Wall time: 111 ms
